# Interactive Configuration Class

This notebook introduces a Python `Settings` class for holding generic configuration data. The structure of the configuration data is specified using a [JSON Schema](http://json-schema.org) file, against which any input or change is validated. The `Settings` class is also capable of creating a GUI to manipulate the configuration within a Jupyter notebook.

To illustrate the use of this class, we will create a fictional configuration for a thermostat with a minimum and maximum temperature, a location string, and a (unspecified) alert function:

In [1]:
schema_json = """
{
  "title": "Thermostat Configuration",
  "type": "object",
  "properties": {
    "min_temp": {
      "description": "Minimum temperature",
      "type": "number",
      "minimum": -30.0,
      "maximum": 150.0,
      "default": 62.0
    },
    "max_temp": {
      "description": "Maximum temperature",
      "type": "number",
      "minimum": -30.0,
      "maximum": 150.0,
      "default": 75.0
    },
    "loc": {
      "description": "Location",
      "type": "string"
    },
    "alert": {
      "description": "Activate alert function",
      "type": "boolean"
    }
  }
}
"""

We can create an empty settings object from this:

In [2]:
from settings import Settings
import json

cfg = Settings(json.loads(schema_json))
cfg

Settings "Thermostat Configuration" {}

We can also show a GUI with the default values (defaults are specified in the schema - if the schema does not provide defaults, reasonable values (i.e. 0, False, or empty strings) are assumed). Note that there should be only one active `gui` object at any time (though you can have multiple views of it).

In [3]:
gui = cfg.interact()
display(gui)

Now, let's load a configuration from another JSON file (note that the GUI updates automatically):

In [4]:
config_json = """
{
        "min_temp": 42.0,
        "max_temp": 69.0,
        "loc": "Building 1",
        "alert": true
}
"""
cfg.from_json(config_json)
cfg

Settings "Thermostat Configuration" {'min_temp': 42.0, 'max_temp': 69.0, 'loc': 'Building 1', 'alert': True}

The configuration elements are accessible like any other collection, and can be used like a dictionary in the application logic to be configured:

In [ ]:
cfg['max_temp']

In [ ]:
cfg['min_temp'] = 33.5

They are also validated according to the underlying JSON schema:

In [ ]:
cfg['min_temp'] = -99.0

## Nested settings objects / tabbing

The `Settings` class also supports recursive nesting. Let's define a schema for a production line consisting of a thermostat and a flow controller:

In [7]:
schema_json = """
{
  "title": "Line Configuration",
  "type": "object",
  "properties": {
    "Temperature": {
      "type": "object",
      "properties": {
        "min_temp": {
          "description": "Minimum temperature",
          "type": "number",
          "minimum": -30.0,
          "maximum": 150.0,
          "default": 62.0
        },
        "max_temp": {
          "description": "Maximum temperature",
          "type": "number",
          "minimum": -30.0,
          "maximum": 150.0,
          "default": 75.0
        },
        "alert": {
          "description": "Activate alert function",
          "type": "boolean"
        }
      }
    },
    "Flow": {
      "type": "object",
      "properties": {
        "min_flow": {
          "description": "Minimum Flow",
          "type": "integer",
          "minimum": -42,
          "maximum": 69,
          "default": 23
        },
        "max_flow": {
          "description": "Maximum Flow",
          "type": "integer",
          "minimum": 0,
          "maximum": 100,
          "default": 42
        }
      }
    },
    "loc": {
      "description": "Location",
      "type": "string"
    }
  }
}
"""

Now, we again create a settings object from the schema:           

In [8]:
cfg = Settings(json.loads(schema_json))
cfg

Settings "Line Configuration" {}

We will also have to modify our initialization data to reflect the hierarchy:

In [9]:
config_json = """
{
    "Temperature": {
        "min_temp": 42.0,
        "max_temp": 69.0,
        "alert": true
    },
    "Flow": {
        "min_flow": 10,
        "max_flow": 88
    },
    "loc": "Area 52"
}"""
cfg.from_json(config_json)
cfg

Settings "Line Configuration" {'loc': 'Area 52'}

Note that there are multiple levels of nesting present. This will create a child settings object for every property declared as an `object`. Every object's widgets will be rendered in a separate tab:

In [10]:
cfg.interact()

We can access the individual values like a multi-level dict:

In [11]:
cfg['Temperature']['max_temp'] = 33.5

Or, alternatively, we can get the child configuration and modify it directly:

In [12]:
temp_cfg = cfg['Temperature']
temp_cfg.interact()

In [13]:
temp_cfg['min_temp'] = -3.5

## TODO

- Populate new Settings objects with defaults from schema
- ~~Support hierarchy of configuration groups (tabs?)~~ - **done**
- Add more special functions to emulate collections completely
- Fix broken defaults parsing for booleans from schema
- ~~Add docstrings!~~ - **done**
- ~~Cache GUI / generate GUI in constructor~~ - **done**
- Read 'Fluent Python' and apply to class